## Motivation
Currently AI is advancing in the field of healthcare to improve detection of malignant tumors, give treatment recommendations, engage patients and support in administrative activities (Davenport and Kalakota 2019). Our goal is to contribute to this field by applying a neural network with transfer learning on a dataset with the aim to detect malignant cells of breast cancer. 

According to Krebsliga Schweiz (2021), there are 6’250 new cases and 1’410 deaths associated with breast cancer in Switzerland every year. Early diagnosis and treatment are a key to increasing the 5-year survival rate of patients.  

From a technical standpoint we want to investigate the performance differences between neural networks with and without transfer learning in the field of tumor detection.

## Data

We use the Kaggle dataset: Breast Histopathology Images, which contains 277’524 images that are classified whether the sample is positive or negative for Invasive Ductal Carcinoma (IDC). Therefore, we face a binary classification problem with this dataset. The sample dataset contains images scanned at 40x zoom that are prepared in 50 x 50-pixel patches.  

In [1]:
import pandas as pd
import numpy as np
from os import listdir
import shutil
from PIL import Image
import tensorflow as tf
import keras
import os
import datetime

from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
# For Image Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout

from tensorflow.keras import layers

In [2]:
tf.__version__

'2.3.0'

In [3]:
tf.keras.__version__

'2.4.0'

In [4]:
from platform import python_version
print(python_version())

3.7.9


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import tensorflow as tf
tf.config.list_physical_devices('GPU')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3066200698734201097
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11325525889116922010
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9988323456
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7027747020207288649
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14191895077087359917
physical_device_desc: "device: XLA_GPU device"
]


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Data preparation
* Initialize the path to all the images
* Create a dataframe with 3 collumns, which store the patient Id, the path to the picture and the picture itself
* Crete the new file strucure 

In [6]:
base_path = "images/IDC_regular_ps50_idx5/"
folder = listdir(base_path)
print("No. of Patients total:",len(folder))

total_images = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = base_path + patient_id
        class_path = patient_path + '/' + str(c) + '/'
        subfiles = listdir(class_path)
        total_images += len(subfiles)
        
print("Total Images in dataset: ", total_images )

No. of Patients total: 279
Total Images in dataset:  277524


Create a pandas data frame to store the patient Id, the path to every patch and the target

In [7]:
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])

k = 0
for n in range(len(folder)):
    patient_id = folder[n]
    patient_path = base_path + patient_id 
    for c in [0,1]:
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            data.iloc[k]["path"] = class_path + image_path
            data.iloc[k]["target"] = c
            data.iloc[k]["patient_id"] = patient_id
            k += 1  

data.head()

,patient_id,path,target
0,10253,images/IDC_regular_ps50_idx5/10253/0/10253_idx...,0
1,10253,images/IDC_regular_ps50_idx5/10253/0/10253_idx...,0
2,10253,images/IDC_regular_ps50_idx5/10253/0/10253_idx...,0
3,10253,images/IDC_regular_ps50_idx5/10253/0/10253_idx...,0
4,10253,images/IDC_regular_ps50_idx5/10253/0/10253_idx...,0


In [8]:
X_data=[]
y_data=[]
for index, row in data[:].iterrows():
    image = Image.open(row['path'])
    npImage = np.asarray(image)
    
    # Resize images with format different than our 50x50 patches
    if npImage.shape != (50, 50, 3):
        image = image.resize((50, 50))
        npImage = np.asarray(image)
    X_data.append(npImage)
    y_data.append(row['target'])
    
    
print('X_data shape: ', np.array(X_data).shape)
print('y_data shape: ', np.array(y_data).shape)

X_data shape:  (277524, 50, 50, 3)
y_data shape:  (277524,)


In [9]:
#Train-validation-test split

x_train,x_test,y_train,y_test=train_test_split(np.asarray(X_data),np.asarray(y_data),test_size=1/6)

x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)

#Dimension of the kaggle dataset
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

input_shape=x_train.shape[1:]
input_shape

((161889, 50, 50, 3), (161889,))
((69381, 50, 50, 3), (69381,))
((46254, 50, 50, 3), (46254,))


(50, 50, 3)

In [10]:
#Image Data Augmentation

train_generator = ImageDataGenerator(rotation_range=20, horizontal_flip=True, vertical_flip=True)

val_generator = ImageDataGenerator(rotation_range=20, horizontal_flip=True, vertical_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)

In [11]:
#Learning Rate Annealer
lrr = ReduceLROnPlateau(monitor='val_accuracy',
                       factor=.1,
                       patience=3,
                       min_lr=1e-8,
                       verbose=1)


#Early stopping callback
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1,
                                      mode='auto', baseline=None, restore_best_weights=True)

In [12]:
name="5x5ConvX2-3x3ConvX2_512DenseWith0.2DropoutX2"
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        
        layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        
        layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),

        layers.Flatten(),
        layers.Dense(1024,activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(1024,activation='relu'),
        layers.Dropout(0.4),

        layers.Dense(1, activation='sigmoid')    
    
    ],name=name
)

In [13]:
model.summary()

Model: "5x5ConvX2-3x3ConvX2_512DenseWith0.2DropoutX2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        4864      
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 64)        102464    
_________________________________________________________________
batch_normalization_1 (Batch (None, 42, 42, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 19, 64)        36928     
_________________________________________________________________
batch_normalization_2 

In [ ]:
batch_size = 128
epochs = 50

model.compile(loss="binary_crossentropy", optimizer=Adam(epsilon=1e-07, learning_rate=0.001), metrics=["accuracy"])

history=model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,callbacks=[es, lrr])

Epoch 1/50
1139/1139 [==============================] - 29s 25ms/step - loss: 0.3921 - accuracy: 0.8414 - val_loss: 0.4829 - val_accuracy: 0.7831
Epoch 2/50
 560/1139 [=============>................] - ETA: 13s - loss: 0.3524 - accuracy: 0.8540

In [ ]:
from tensorflow import keras
from tensorflow.keras.regularizers import l2

input = tf.keras.Input(shape=input_shape)
beforeModel = tf.keras.layers.UpSampling2D()(input)
beforeModel = tf.keras.layers.UpSampling2D()(beforeModel)
beforeModel = tf.keras.layers.Reshape((224,224), input_shape(200, 200))
#beforeModel = tf.keras.layers.UpSampling2D()(beforeModel)
print(beforeModel)
resnet = tf.keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))
resnet.trainable=False

x = resnet(beforeModel,training=False)
x = keras.layers.Flatten()(x)

x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(1024,
                       kernel_regularizer=l2(0.001),
                       bias_regularizer=l2(0.001),
                       activation='relu')(x) # dense layer 1 
'''
x = keras.layers.Dropout(0.3)(x)

x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(1024,
                       activation='relu',
                       kernel_regularizer=l2(0.001),
                       bias_regularizer=l2(0.001))(x) # dense layer 2
x = keras.layers.Dropout(0.2)(x) 
'''

output = keras.layers.Dense(units=1, activation='sigmoid')(x)

In [ ]:
model = keras.Model(inputs = input, outputs = output)
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(epsilon=0.1, learning_rate=0.001), metrics=["accuracy"])
log_dir= os.path.join('logs','ResNet50',datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),'')

# tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history=model.fit(train_generator.flow(x_train, y_train),
                  batch_size=batch_size, epochs=epochs,
                  validation_data=val_generator.flow(x_val, y_val),
                  callbacks=[es, lrr]) 